In [1]:
import operator
import pandas as pd
import pickle
import random
import snap

In [2]:
def get_comm_info(comm_file):
    '''
    get community information, two maps
    map1: key: user id, value: community id array
    map2: key: community id, value: user id array
    '''
    comm_map_usr = {}
    comm_map_comm = {}
    comm_id = 0
    with open(comm_file, 'r') as cf:
        for line in cf:
            node_list = line.split('\t')
            node_list = [int(id) for id in node_list]
            for id in node_list:
                if id in comm_map_usr:
                    comm_map_usr[id].append(comm_id)
                else:
                    comm_map_usr[id] = [comm_id]
            comm_map_comm[comm_id] = node_list[:]
            comm_id += 1
    return comm_map_usr, comm_map_comm

In [2]:
gf_file = "data/com-lj.ungraph.txt"
gf = snap.LoadEdgeList(snap.PUNGraph, gf_file, 0, 1)

In [3]:
print "Load graph! With nodes ", gf.GetNodes(), " and edges ", gf.GetEdges()

Load graph! With nodes  3997962  and edges  34681189


In [4]:
##--get graph map
def get_graph_info(gf_file):
    '''
    get graph information as a map
    key: user id, value: list of friend id
    '''
    gf_map = {}
    with open(gf_file, 'r') as cf:
        for line in cf:
            if line[0] != '#':
                node_list = line.split('\t')
                if len(node_list) == 2:
                    node_list = [int(id) for id in node_list]
                    if node_list[0] in gf_map:
                        gf_map[node_list[0]].add(node_list[1])
                    else:
                        gf_map[node_list[0]] = set([node_list[1]])
                    if node_list[1] in gf_map:
                        gf_map[node_list[1]].add(node_list[0])
                    else:
                        gf_map[node_list[1]] = set([node_list[0]])                    
                else:
                    print "wrong format line: ", line
    return gf_map

In [5]:
gf_map = get_graph_info(gf_file)

In [7]:
len(gf_map)

3997962

In [12]:
def lnr_thres_mod_1step_v2(gf_map, act_nod_list, new_act_nod, nd_prob_map, nbr_map):
    '''
    perform 1 step linear threshold model model
    '''
    new_pre_nod = set()
    for id in new_act_nod:
        for nbr_id in gf_map[id]:
            if nbr_id not in act_nod_list:
                if nbr_id not in nbr_map:
                    count = 0
                    for nbr_nbr_id in gf_map[nbr_id]:
                        if nbr_nbr_id in act_nod_list:
                            count += 1
                    nbr_map[nbr_id] = count
                else:
                    nbr_map[nbr_id] += 1
                if nbr_id not in nd_prob_map:
                    nd_prob_map[nbr_id] = random.randint(1, len(gf_map[nbr_id]))
                if nbr_map[nbr_id] >= nd_prob_map[nbr_id]:
                    new_pre_nod.add(nbr_id)
                    act_nod_list.add(nbr_id)
                    nd_prob_map.pop(nbr_id)
                    nbr_map.pop(nbr_id)
    return act_nod_list, new_pre_nod

In [14]:
##--get run the model with random initial
prob = 0.01
mean_list = {}
std_list = {}

for total_iter in [10]: #[10, 100, 1000, 10000, 100000, 1000000]:
    total_influence = [0] * total_iter
    for i in xrange(total_iter):
        init_set = 1
        new_act_nod = set()
        nd_prob_map = {}
        nbr_map = {}
        for j in xrange(init_set):
            new_act_nod.add(gf.GetRndNId())
        act_nod_list = new_act_nod.copy()
        count = 0
        while len(new_act_nod) != 0:
            count += 1
            act_nod_list, new_act_nod = lnr_thres_mod_1step_v2(gf_map, act_nod_list, new_act_nod, nd_prob_map, nbr_map)
            if i % (total_iter / 10) == 0:
                print total_iter, i, len(act_nod_list), len(new_act_nod)
        total_influence[i] = len(act_nod_list)
    total_influence = pd.DataFrame(total_influence)
    mean_list[total_iter] = total_influence.mean()[0]
    std_list[total_iter] = total_influence.std()[0]
    print total_iter, mean_list[total_iter], std_list[total_iter]

1
{2642784: 35, 2642849: 20, 2092835: 6, 2092837: 2, 1064361: 26, 2092829: 14, 1898419: 14, 2092825: 40, 2092828: 48, 1064349: 14}
{2642784: 1, 2642849: 1, 2092835: 1, 2092837: 1, 1064361: 1, 2092829: 1, 1898419: 1, 2092825: 1, 2092828: 1, 1064349: 1}
set([])
1
{2520037: 10, 377832: 22, 1207881: 32, 917201: 164, 475858: 32, 3462709: 5}
{2520037: 1, 377832: 1, 1207881: 1, 917201: 1, 475858: 1, 3462709: 1}
set([])
1
{1504513: 6, 1585284: 3, 1969285: 18, 206984: 98, 293514: 41, 1145687: 29, 49421: 8, 599055: 37, 48785: 229, 1054610: 7, 411028: 4, 340379: 7, 272346: 21, 279967: 26, 205360: 136, 620323: 25, 337350: 29, 30325: 86, 422277: 6, 558761: 7, 347693: 7, 407093: 55, 8112: 121, 382257: 137, 460467: 27, 379573: 37, 1733305: 16, 939706: 25, 64188: 91, 2902973: 3, 151870: 2, 33909: 51, 407088: 25, 1252164: 33, 409286: 76, 1048865: 29, 791752: 20, 202569: 57, 362956: 22, 362957: 31, 362958: 43, 492496: 9, 974290: 7, 32467: 16, 596820: 8, 573397: 24, 588759: 10, 549720: 54, 1008987: 5, 10

In [30]:
mean_list

{1: 1.0, 10: 1.1000000000000001}

In [31]:
std_list

{1: nan, 10: 0.31622776601683794}

In [28]:
total_influence.mean()[0]

1.2

In [16]:
226./328

0.6890243902439024